In [91]:
# --- Matrix Class --- 

from Array import Array2D

class Matrix(Array2D):

    def __init__(self,numRows,numCols):
        super().__init__(numRows,numCols)
        self._shape = numRows , numCols

    # Multiple Scalar 
    def scaleBy(self,scalar):

        newMatrix = Matrix(self.numRows() , self.numCols())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[i,j] = self[i,j] * scalar
        return newMatrix

    # Square all data
    def square(self):
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                self[i,j] = self[i,j] * self[i,j]

    # Transpose สลับแถวกับหลัก
    def transpose(self):
        newMatrix = Matrix(self.numCols() , self.numRows())
        for i in range( self.numRows()):
            for j in range( self.numCols()):
                newMatrix[j,i] = self[i,j]
        return newMatrix

    # Addition
    def __add__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] + rhsMatrix[i,j]
        return newMatrix
        
    # Subtraction
    def __sub__ ( self , rhsMatrix ) :
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows() , self.numCols() )
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] - rhsMatrix[i,j]
        return newMatrix
    
    # Element-wise Multiplication
    def __mul__ ( self , rhsMatrix):
        assert rhsMatrix.numRows() == self.numRows() and rhsMatrix.numCols() == self.numCols() ,\
            "Matrix size not compatible for the add operation."
        newMatrix = Matrix( self.numRows(), self.numCols())
        for i in range( self.numRows() ):
            for j in range( self.numCols() ) :
                newMatrix[i,j] = self[i,j] * rhsMatrix[i,j]
        return newMatrix    

    # Dot Product
    def dotproduct(self,rhsMatrix):
        assert self.numCols() == rhsMatrix.numRows() , "Cannot multiply these matrices."
        newMatrix = Matrix( self.numRows() , rhsMatrix.numCols())
        for i in range(self.numRows()):
            for j in range(rhsMatrix.numCols()):
                total = 0
                for k in range(rhsMatrix.numRows()):
                     total += self[i,k] * rhsMatrix[k,j]
                newMatrix[i,j] = total
        return newMatrix

    #Load list
    def loadList(self,data):
        # Check that they has 1 column or more. 
        data_column = len(data[0])
        data_row = len(data)
            
        assert (self.numCols() == data_column and self.numRows() == data_row) , "incorrect size to load data to matrix."
        for i in range(data_row):
            for j in range(data_column):
                self[i,j] = data[i][j]

    def reshape(self,row,col):
        
        # เช็คว่าขนาดในการ Reshape เท่ากันไหม
        assert (row * col == self.numRows() * self.numCols()) , "Size couldn't fit in the Array2D"
        
        # 1 Solution - เข้าถึง matrix ก่อน แล้วดึงเข้า list เปล่า
        number_list = []
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                number_list.append(self[i,j])

        # 2 Solution - เข้าถึง Matrix รูปร่างใหม่ แล้ว Assign ค่าลงไปผ่าน list เดี่ยว (ใช้ index บวกไปเรื่อยๆ)
        
        newMatrix = Array2D(row,col)
        ndx = 0

        for i in range(row):
            for j in range(col):
                newMatrix[i,j] = number_list[ndx]
                ndx += 1

        return newMatrix
    
    # Load Function - Let every data in hidden goes through the activate function.
    def loadFunction(self,activate_function):        
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                self[i,j] = activate_function(self[i,j])

    # Average - compute the weighted average of all the data
    def average(self):
        total = 0
        for i in range(self.numRows()):
            for j in range(self.numCols()):
                total += self[i,j]

        average = total / (self.numRows() * self.numCols())
        return average
                
    # Trace Method (ผลบวกทแยงมุม)

    # Argmax
    def argmax_zero_axis(self):

        ''' 
            Input :
            [ [0... 30],
              [0... 30],
              [1... 30]
            ]
            แนวตั้งจะแทน ตัวข้อมูลนึง เช่น 001 , 010 , 100
        '''
        index = []
        print(self)
        for j in range(self.numCols()):

            # กำหนดให้ตัวแถวแรกเป็น max_value
            max_value = self[0, j]
            max_index = 0
            for i in range(self.numRows()):
                print(self[i,j])
                if max_value < self[i, j]:
                    max_value = self[i,j]
                    max_index = i
            else:
                index.append(max_index)

        return index
                    
        

    

In [18]:
# --- ANN Model Class ---
import random
import math
import statistics

class ANN_Model():
    def __init__(self,file): 
        with open(file,'r') as f:
            self._RawData = f.readlines()

    def cleanData(self):

        newData = []
        resultData = []
        for row in self._RawData:
            check_row = row.split(',')
            last = check_row.pop()
            resultData.append(last[0])


            check_row = [float(num) for num in check_row]

            # Column
            current_row = []
            for column in range(len(check_row)):
                if (column != 1):
                    current_row.append(check_row[column])
  
            newData.append(current_row)

        # self.Normalization(newData,resultData)
        self._RawInputData = newData

        # result Data is ['g','b','g',...]
        rawtargetdata = [list(result) for result in resultData]
        ## We can detect it [1,0] ,[0,1]

        self._RawTargetData = rawtargetdata

        rawCleanInputData = []
        rawCleanTargetData = []

        # Remove the row that is the suspicious data
        for i in range(len(self._RawInputData)):

            okay_flag = True

            if self._RawInputData[i][0] == 0:
                okay_flag = False
            
            for j in range(len(self._RawInputData[0])):
                if self._RawInputData[i][j] % 1 != 0: # Check if it is all integer. if it floats just true
                    break
                
            else:
                okay_flag = False

            if (okay_flag == True):
                rawCleanInputData.append(self._RawInputData[i])
                rawCleanTargetData.append(self._RawTargetData[i])
                
        self._RawCleanInputData = rawCleanInputData
        self._RawCleanTargetData = rawCleanTargetData
        

        # Create instant variables for hyperparameter
        X = self._RawCleanInputData
        y = self._RawCleanTargetData

        self._RawCleanData = []

        # Merge and write it in the csv file to check

        for i in range(len(self._RawCleanInputData)):
            self._RawCleanData.append(self._RawCleanInputData[i] + self._RawCleanTargetData[i])

        import csv
        

        with open('rawcleandata.csv' , 'w' , encoding="UTF-8" , newline="") as f:
            writer = csv.writer(f)

            # write multiple rows
            writer.writerows(self._RawCleanData)

        self._NumData = len(X)
        self._NumInput = len(X[0])
        self._NumOutput = len(y[0])

    # Rescaling Min-Max Normalization

    def StandardScaler(self):

        # Evaluate the mean and variance of all inputs

        all_input_data = []
        for i in range(len(self._RawCleanInputData)):
            for j in range(len(self._RawCleanInputData[0])):
                all_input_data.append(self._RawCleanInputData[i][j])

        # Standard Scaler = x(i) - mean / S.D -> S.D = Variance**2

        mean = statistics.mean(all_input_data)

        stdev = statistics.stdev(all_input_data)

        print("Mean : " , mean)
        print("S.D. : " , stdev)

        Scaled_Train_Data = []
        for i in range(len(self._RawCleanInputData)):

            scaled_data = []
            for j in range(len(self._RawCleanInputData[0])):
                newScale = (self._RawCleanInputData[i][j] - mean) / stdev
                scaled_data.append(newScale)

            Scaled_Train_Data.append(scaled_data)
        
        # List Concatetrating

        for i in range(len(Scaled_Train_Data)):
            Scaled_Train_Data[i] = Scaled_Train_Data[i] + self._RawCleanTargetData[i]

        self._Scaled_Data = Scaled_Train_Data
        self._Scaled_Data.sort()

        # random.shuffle(self._Scaled_Data)

        # print(self._Scaled_Data)

        # You can shuffle there after sorting

        self._ScaledTrainData = []
        self._ScaledTargetData = []

        for column in self._Scaled_Data:
            self._ScaledTrainData.append(column[:-2])
            self._ScaledTargetData.append(column[-2:])

    def MinMax_Normalization(self):
        Scaled_Train_Data = []
        for column in self._RawCleanInputData:
            scaled_data = []
            min_vals = min(column)
            max_vals = max(column)
            for value in column:
                newScale = (value - min_vals) / (max_vals - min_vals)
                scaled_data.append(newScale)

            Scaled_Train_Data.append(scaled_data)

        # List Concatetrating

        for i in range(len(Scaled_Train_Data)):
            Scaled_Train_Data[i] = Scaled_Train_Data[i] + self._RawCleanTargetData[i]

        self._Scaled_Data = Scaled_Train_Data
        self._Scaled_Data.sort()

        # random.shuffle(self._Scaled_Data)

        # print(self._Scaled_Data)

        # You can shuffle there after sorting

        self._ScaledTrainData = []
        self._ScaledTargetData = []

        for column in self._Scaled_Data:
            self._ScaledTrainData.append(column[:-2])
            self._ScaledTargetData.append(column[-2:])

    # Data Spliting
    def splitData(self, k=5):

        self._Train_X , self._Train_y , self._Test_X , self._Test_y = [] , [] , [] , []
        
        # K-fold Cross Validation โดย k เป็น 5

        data = list(zip(self._ScaledTrainData, self._ScaledTargetData))
    
        # Randomly shuffle the data
        random.shuffle(data)

        fold_size = len(data) // k

        for i in range(k):
            # Assign a fold for testing
            test_data = data[i * fold_size : (i + 1) * fold_size]
            test_X, test_y = zip(*test_data)
            
            # The rest of the data is for training
            train_data = data[:i * fold_size] + data[(i + 1) * fold_size:]
            train_X, train_y = zip(*train_data)

            self._Test_X.append(test_X)
            self._Test_y.append(test_y)
            self._Train_X.append(train_X)
            self._Train_y.append(train_y)


        # X = self._ScaledTrainData
        # y = self._ScaledTargetData


        # for i in range(len(self._ScaledTrainData)):
        #     if (i%5 == 4):
        #         self._Test_X.append(X[i])
        #         self._Test_y.append(y[i])
        #     else:
        #         self._Train_X.append(X[i])
        #         self._Train_y.append(y[i])

        self._NumTrain = len(self._Train_X)
        self._NumTest = len(self._Test_X)

        Input_Train = Matrix(len(self._Train_X),len(self._Train_X[0]))
        Input_Train.loadList(self._Train_X)
        # Input in matrix form
        self._Input_Train = Input_Train.transpose()

        Output_Train = Matrix(len(self._Train_y),len(self._Train_y[0]))
        Output_Train.loadList(self._Train_y)
        # Output in matrix form
        self._Output_Train = Output_Train.transpose()

        Input_Test = Matrix(len(self._Test_X), len(self._Test_X[0]))
        Input_Test.loadList(self._Test_X)
        
        self._Input_Test = Input_Test.transpose()

        Output_Test = Matrix(len(self._Test_y), len(self._Test_y[0]))
        Output_Test.loadList(self._Test_y)

        self._Output_Test = Output_Test.transpose()

    def hyperparameter(self):
        # Rules of Thumbs
        self._NumHidden = int((2/3) * (self._NumInput + self._NumOutput))
        print("No.Input Unit: ",self._NumInput)
        print("No.Hidden Layer: ", 1)
        print("No.Hidden Unit: ",self._NumHidden)
        print("No.Output Unit: ",self._NumOutput)
    
    def trainModel(self,random_state=None,epochs=50,alpha=0.01):

        # กำหนด Random Seed
        if random_state is not None:
            random.seed(random_state)
            print("Random Seed :" , random_state)
        else: 
            random.seed(42)
            print("Random Seed :" , 42)

        # Random weights between input and hidden

        # --- Uniform Weight Initialization 
        
        self._weights_input_hidden = Matrix(self._NumInput , self._NumHidden)
        for i in range(self._NumInput):
            for j in range(self._NumHidden):
                self._weights_input_hidden[i,j] = random.uniform(-1,1)

        # Random weights between hidden and output

        self._weights_hidden_output = Matrix(self._NumHidden , self._NumOutput)
        for i in range(self._NumHidden):
            for j in range(self._NumOutput):
                self._weights_hidden_output[i,j] = random.uniform(-1,1)

        # # Xavier/Godot Random weights between input and hidden
        # self._weights_input_hidden = Matrix(self._NumInput, self._NumHidden)
        # for i in range(self._NumInput):
        #     for j in range(self._NumHidden):
        #         limit = math.sqrt(2 / self._NumInput)
        #         self._weights_input_hidden[i, j] = random.uniform(-limit, limit)

        # # # Random weights between hidden and output
        # self._weights_hidden_output = Matrix(self._NumHidden, self._NumOutput)
        # for i in range(self._NumHidden):
        #     for j in range(self._NumOutput):
        #         limit = math.sqrt(2 / self._NumHidden)
        #         self._weights_hidden_output[i, j] = random.uniform(-limit, limit)

        Input = self._Input_Train
        Target = self._Output_Train

        losses = []

        # Sigmoid - Activate Function 
        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        # ReLu - Activate Function
        def relu(x):
            return max(0, x)
        
        def tanh(x):
            return (math.exp(2*x) - 1) / (math.exp(2*x) + 1)

        # Training Loop
        for epoch in range(epochs):

            random_list = [int(i) for i in range(self._NumTrain)]
            Train_seq = random.sample(random_list,self._NumTrain)

            for seq_num in range(self._NumTrain):

                # Select Column ([:,Train_seq[i]])
                input_selected_column = []
                for i in range(Input.numRows()):
                    for j in range(Input.numCols()):
                        if (j == Train_seq[seq_num]):
                            input_selected_column.append(Input[i,j])

                input_selected_column_Matrix = Matrix(1,len(input_selected_column))
                for i in range(len(input_selected_column)):
                    input_selected_column_Matrix[0,i] = input_selected_column[i]
                input_selected_column_Matrix = input_selected_column_Matrix.transpose()

                # Feed Forward

                Hidden = self._weights_input_hidden.transpose().dotproduct(input_selected_column_Matrix)

                Hidden.loadFunction(sigmoid)

                Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
                Output.loadFunction(sigmoid)

                # --- Backpropagation Algorithm
                target_selected_column = []
                for i in range(Target.numRows()):
                    for j in range(Target.numCols()):
                        if (j == Train_seq[seq_num]):
                            target_selected_column.append(Target[i,j])

                # Target Select Column - 
                target_selected_column_Matrix = Matrix(1,len(target_selected_column))
                for i in range(len(target_selected_column)):
                    target_selected_column_Matrix[0,i] = target_selected_column[i]
                target_selected_column_Matrix = target_selected_column_Matrix.transpose()
                Error = target_selected_column_Matrix - Output
                # Without no scaling factor
                Error.scaleBy(1/2)

                Error_wrt_output = target_selected_column_Matrix - Output
                Error_wrt_output.scaleBy(-1)

                # oneminus_Output = 1 - Output
                oneminus_Output = Matrix(Output.numRows(),Output.numCols())
                oneminus_Output.clear(1) 
                Output_wrt_outputLayer_LinearTransform = Output * (oneminus_Output - Output)
                OutputLayer_LinearTransform_wrt_weights_hidden_output = Hidden
                Error_wrt_weights_hidden_output = OutputLayer_LinearTransform_wrt_weights_hidden_output.dotproduct((Error_wrt_output * Output_wrt_outputLayer_LinearTransform).transpose())
                
                # oneminus_Hidden = 1 - Hidden
                oneminus_Hidden = Matrix(Hidden.numRows(),Hidden.numCols())
                oneminus_Hidden.clear(1)
                HiddenLayer_activations_wrt_hiddenLayer_linearTransform = Hidden * (oneminus_Hidden - Hidden)
                OutputLayer_LinearTransform_wrt_hiddenLayer_activations = self._weights_hidden_output
                HiddenLayer_linearTransform_wrt_weights_input_hidden = input_selected_column_Matrix
                Error_wrt_weights_input_hidden = HiddenLayer_linearTransform_wrt_weights_input_hidden.dotproduct(\
                    (HiddenLayer_activations_wrt_hiddenLayer_linearTransform *\
                     (OutputLayer_LinearTransform_wrt_hiddenLayer_activations\
                    .dotproduct (Output_wrt_outputLayer_LinearTransform * Error_wrt_output))).transpose())
                
                # # L2 regularization term for hidden to output weights
                # l2_hidden_output = self._weights_hidden_output.scaleBy(lmbda)

                # # L2 regularization term for input to hidden weights
                # l2_input_hidden = self._weights_input_hidden.scaleBy(lmbda)

                # # Update weights with regularization terms
                # self._weights_hidden_output = self._weights_hidden_output - (
                #     Error_wrt_weights_hidden_output.scaleBy(alpha) + l2_hidden_output
                # )
                # self._weights_input_hidden = self._weights_input_hidden - (
                #     Error_wrt_weights_input_hidden.scaleBy(alpha) + l2_input_hidden
                # )

                # #Normal Updating weights term
                self._weights_hidden_output = self._weights_hidden_output - Error_wrt_weights_hidden_output.scaleBy(alpha)
                self._weights_input_hidden = self._weights_input_hidden - Error_wrt_weights_input_hidden.scaleBy(alpha)

            epoch_loss = Error.average()
            if epoch % 10 == 0:
                print(f"Error at epoch {epoch} is {epoch_loss:.5f}")

            losses.append(epoch_loss)

        # Visualizing the error
        import matplotlib.pyplot as plt
        arange = [int(i) for i in range(1,epochs+1)]
        plt.plot(arange, losses)

    def testModel(self):
        Input = self._Input_Test
        Target = self._Output_Test

        def sigmoid(x): # หรือ Logistic Function
            return 1 / (1 + math.exp(-x))
        
        def relu(x):
            return max(0, x)
        
        def tanh(x):
            return (math.exp(2*x) - 1) / (math.exp(2*x) + 1)

        # Forward Propogation

        Hidden = self._weights_input_hidden.transpose().dotproduct(Input)
        Hidden.loadFunction(sigmoid)

        # Calculating the output
        Output = self._weights_hidden_output.transpose().dotproduct(Hidden)
        Output.loadFunction(sigmoid)

        Class_Output = Output.argmax_zero_axis()
        self._class_output = Class_Output
        print("Class_Output : ")
        print(Class_Output)
        Class_Target = Target.argmax_zero_axis()
        self._class_target = Class_Target
        # print("Class_Target : ")
        # print(Class_Target)
        correct = 0
        miss = 0

        conf_matrix = Matrix(self._NumOutput , self._NumOutput)
        conf_matrix.clear(0)
        for i in range(self._NumTest):
            if Class_Output[i] == Class_Target[i]:
                correct += 1
            else:
                miss += 1
            conf_matrix[Class_Target[i],Class_Output[i]] += 1
                
        print('Confusion Matrix (row=Actual, col=Predicted)')
        print(conf_matrix)
        print('accuracy = {0:.2f}%'.format(correct/self._NumTest))

In [19]:
import os

os.getcwd()

'c:\\Users\\folks\\Desktop\\cpe112-data-structure\\Final_Project'

In [21]:
# --- 2. Data preprocessing

myModel = ANN_Model('ionosphere_data.csv')
myModel.cleanData()

In [94]:
myModel._RawInputData

[[1.0,
  0.99539,
  -0.05889,
  0.85243,
  0.02306,
  0.83398,
  -0.37708,
  1.0,
  0.0376,
  0.85243,
  -0.17755,
  0.59755,
  -0.44945,
  0.60536,
  -0.38223,
  0.84356,
  -0.38542,
  0.58212,
  -0.32192,
  0.56971,
  -0.29674,
  0.36946,
  -0.47357,
  0.56811,
  -0.51171,
  0.41078,
  -0.46168,
  0.21266,
  -0.3409,
  0.42267,
  -0.54487,
  0.18641],
 [1.0,
  1.0,
  -0.18829,
  0.93035,
  -0.36156,
  -0.10868,
  -0.93597,
  1.0,
  -0.04549,
  0.50874,
  -0.67743,
  0.34432,
  -0.69707,
  -0.51685,
  -0.97515,
  0.05499,
  -0.62237,
  0.33109,
  -1.0,
  -0.13151,
  -0.453,
  -0.18056,
  -0.35734,
  -0.20332,
  -0.26569,
  -0.20468,
  -0.18401,
  -0.1904,
  -0.11593,
  -0.16626,
  -0.06288,
  -0.13738],
 [1.0,
  1.0,
  -0.03365,
  1.0,
  0.00485,
  1.0,
  -0.12062,
  0.88965,
  0.01198,
  0.73082,
  0.05346,
  0.85443,
  0.00827,
  0.54591,
  0.00299,
  0.83775,
  -0.13644,
  0.75535,
  -0.0854,
  0.70887,
  -0.27502,
  0.43385,
  -0.12062,
  0.57528,
  -0.4022,
  0.58984,
  -0.22145,

In [95]:
myModel._RawTargetData

[[1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 

In [96]:
myModel.StandardScaler()

Mean :  0.2722591259765625
S.D. :  0.5599094378749263


In [97]:
myModel._ScaledTargetData

[[0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 

In [98]:
#myModel.MinMax_Normalization()

In [99]:
len(myModel._ScaledTrainData)

320

In [100]:
# --- 3. Split Data --- 

myModel.splitData()

[([1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [1, 0], [1, 0]

In [101]:
myModel._Input_Train

[ [1.2997474677074503, 0.9398142599999958, 0.19715844484139028, 0.9569063098077681, -0.7892332153816539, 1.2997474677074503, -0.6229384653710297, 1.2667242701164674, -0.6183841574285198, 0.7678221600534475, -0.11721382340981873, 1.2951931597649404, -0.4486602814376529, 1.2678494520787345, -0.4418377495394617, 1.2997474677074503, -0.7197576942194449, 1.222288512622489, -0.3484297866401421, 1.2997474677074503, -0.4828443810531974, 1.1266123257674878, 0.06390475245289631, 1.2997474677074503, -0.7459583599122368, 1.2997474677074503, -0.4932924992742494, 0.8646413889018628, -0.34501852069104655, 1.2997474677074503, -0.8211312310103699, 1.2997474677074503], [1.2997474677074503, 1.1864791501725584, 0.1407564664788566, 0.8577831369413773, 0.6860767974931825, 0.32764383240208417, 1.0957144718830094, -0.21258638973531904, 1.2243066961421107, -0.8751578252303397, 1.233683212494337, -1.4959725079034467, 1.0032173705721918, -1.892750959867363, 0.5504298609309776, -2.1490959869216524, -0.02153763655

In [102]:
print("Input in matrix form :")
print(myModel._Input_Train)

print("\nShape of Input Matrix:" , myModel._Input_Train._shape)

Input in matrix form :
[ [1.2997474677074503, 0.9398142599999958, 0.19715844484139028, 0.9569063098077681, -0.7892332153816539, 1.2997474677074503, -0.6229384653710297, 1.2667242701164674, -0.6183841574285198, 0.7678221600534475, -0.11721382340981873, 1.2951931597649404, -0.4486602814376529, 1.2678494520787345, -0.4418377495394617, 1.2997474677074503, -0.7197576942194449, 1.222288512622489, -0.3484297866401421, 1.2997474677074503, -0.4828443810531974, 1.1266123257674878, 0.06390475245289631, 1.2997474677074503, -0.7459583599122368, 1.2997474677074503, -0.4932924992742494, 0.8646413889018628, -0.34501852069104655, 1.2997474677074503, -0.8211312310103699, 1.2997474677074503], [1.2997474677074503, 1.1864791501725584, 0.1407564664788566, 0.8577831369413773, 0.6860767974931825, 0.32764383240208417, 1.0957144718830094, -0.21258638973531904, 1.2243066961421107, -0.8751578252303397, 1.233683212494337, -1.4959725079034467, 1.0032173705721918, -1.892750959867363, 0.5504298609309776, -2.149095986

In [103]:
print("Output in matrix form:")
print(myModel._Output_Train)

print("\nShape of Output Matrix:" , myModel._Output_Train._shape)

Output in matrix form:
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [0, 1], [0, 1], [0, 1], [0, 1] ]
[ [1, 0], [0, 1], [0, 1], [0, 1], [0, 1] ]
[ [1, 0], [0, 1], [0, 1], [0, 1], [0, 1] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [0, 1], [0, 1], [0, 1], [0, 1] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [0, 1], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [1, 0], [1, 0], [1, 0], [1, 0] ]
[ [1, 0], [0, 1], [0, 1], [0, 1], [0, 1] ]
[ [0, 1], [0, 1], [0, 1], [0, 1

### 4. กำหนด Hyperparameter

* Hidden Unit จะมีค่าเท่ากับ 2/3 ของ Input + Output ตามหลัก Rule of Thumbs

In [104]:
myModel.hyperparameter()

No.Input Unit:  32
No.Hidden Layer:  1
No.Hidden Unit:  22
No.Output Unit:  2


### 5. เริ่มกระบวนการ Train

In [105]:
myModel.trainModel(7,50,0.01)

Random Seed : 7


AssertionError: Cannot multiply these matrices.

In [ ]:
myModel._weights_input_hidden

[ -0.3178236247820637, -0.7359571539062083, 0.12310599119533884, -0.9680350561374886, 0.22069261716809982, 0.08579162832006791, -0.9553226080460706, 0.07036401175204696, -0.7479049667778035, 0.04422236718435242, -0.6454153935517545, -0.7317598151666881, 0.08339004071680585, 0.792871755675324, -0.7297627608284601, -0.7796824914153003, 0.6710460758110127, 0.805235574672378, 0.058977317707282456, -0.2091786290620094, 1.0424162205310838, -0.8373834561850471 ]
[ 0.7535272470540416, -0.45655273261266116, -0.8821498074168233, -0.9080943593889225, -0.15579498197187303, 0.935357796131415, -0.7250337732611326, 0.25699553284990234, 0.41797891465698617, -0.08200666687751565, 0.27466712186319964, -0.8044370330254236, -0.6954195792370864, -0.47670875917401484, 0.39677054266199946, -0.31557232561882576, 0.016894286358291898, 0.0888856217515314, -0.1877153108885245, -0.3938303798821414, 0.6690485872182915, 0.45073476834423465 ]
[ -0.5079760039058, 0.15290548359410852, 0.04937847467945863, 0.7232781309

In [ ]:
myModel._weights_hidden_output

[ -0.19361416657530556, 1.200835607294417 ]
[ -0.09022971543323412, -0.2993830408516181 ]
[ 0.7396777902617414, -0.15877324505068538 ]
[ 0.5751017031093764, -0.42415090523995047 ]
[ -0.8337917846852309, 0.968800946621234 ]
[ -0.7924107280997285, 1.4833163952370578 ]
[ -0.024274033935691254, -0.4749520215807137 ]
[ -1.1150261351075001, 0.5455968989483414 ]
[ -0.4947391878236803, 1.6053230712639142 ]
[ -1.0795316073669114, 0.20071471136041627 ]
[ -1.0018222794892482, 1.3306790877479093 ]
[ -1.2579638880187158, -0.4410772938215044 ]
[ -1.2486193319399337, 0.14510159332303946 ]
[ 0.24311418640792032, 1.4826441435827022 ]
[ 0.18586652175703747, 1.2118475440294254 ]
[ 0.8065728524465122, -0.20047355635573946 ]
[ -0.9579824426286885, 1.30535658658463 ]
[ -0.23355692216680018, -0.23655210058890053 ]
[ 0.21309717048575455, -0.07719334879015069 ]
[ -0.3167007063694732, -0.07555200422396421 ]
[ -1.4389820293565916, -0.17290456676400176 ]
[ -0.7284553448238237, 0.16727116418085503 ]

In [ ]:
myModel.testModel()

[ 0.040536290414459476, 0.0054589173458992005, 0.014212376033208093, 0.02809259649371546, 0.01576964910423187, 0.008023583358075926, 0.0012152533246360256, 0.02070774644513238, 0.005489147063358657, 0.003208945383577759, 0.008024576583784951, 0.0016639650015266817, 0.005006815308678024, 0.0011378566917115377, 0.00159606917539736, 0.0019296267442304915, 0.0021532036596860404, 0.0007498607910960102, 0.0009381244718380823, 0.0009139495517091955, 0.01178461413610698, 0.001041877606567783, 0.020695630715931033, 0.013884097621752068, 0.000763461406767165, 0.0013377963509735979, 0.0019336521677110615, 0.0009334996338186524, 0.00040801916480760184, 0.0007332511866741863, 0.000531547349336906, 0.0022041938654152045, 0.00042641973779963165, 0.0007922942170260087, 0.002482407081100107, 0.0025812421268326087, 0.0014043216929180227, 0.0026076967160795906, 0.001804416135584286, 0.002339181150557706, 0.00040914218615955466, 0.028751466897044064, 0.0005267900628273387, 0.0038252775833262378, 0.0006914

In [ ]:
myModel._class_output

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
myModel._class_target

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]